# Files documentation
The workflow here is to have this notebook to describe in more details the files that are available. For version control
purposes this file should be commited without output and only run locally.

In [1]:
from pathlib import Path
from pprint import pprint

import numpy as np
import scipy as sp
import pandas as pd
import h5py
import mat73
from scipy.io import loadmat

# Data loading
Here we load our base path:

In [21]:
data_location = '/home/heberto/globus_data'  # Change this with the right location
data_path = Path(data_location)
author_path = Path("SenzaiY")
base_path = data_path.joinpath(author_path)

Now, this data sets is organized with one folder per subject. Let's peak inside of  `base_path`: 

In [22]:
subject_path_dic = {p.stem:p for p in base_path.iterdir() if p.is_dir()}
subject_path_dic.keys()

dict_keys(['YMV01', 'YMV04'])

The output should be something like ['YMV01', 'YMV02', ...] indicating the different subjects

Inside each of the subjects we can find a folder per sesion:

In [28]:
subject = 'YMV01'
sessions_path_dic = {p.stem:p for p in subject_path_dic[subject].iterdir() if p.is_dir()}
sessions_path_dic.keys()

dict_keys(['YMV01_170818'])

The ouput of this should be: `YMV01_170818`. 

The name of the sessions fits the following pattern `{subject}_{date}`.

Let's gather all the available sessions in one dic for convenience

In [33]:
data_path = Path("/home/heberto/globus_data")
author_path = Path("SenzaiY")
base_path = data_path.joinpath(author_path)

session_list = [
    session
    for subject in base_path.iterdir()
    if subject.is_dir() and "YMV" in subject.name
    for session in subject.iterdir()
]
session_path_dic = {session.stem:session for session in session_list if session.is_dir()}
session_path_dic

{'YMV01_170818': PosixPath('/home/heberto/globus_data/SenzaiY/YMV01/YMV01_170818'),
 'YMV04_170907': PosixPath('/home/heberto/globus_data/SenzaiY/YMV04/YMV04_170907')}

The output here should be a combination of session:path for all the sessions

# An overview of the available data
Let's find out which data types are available. The files with formats `.jpg`, `.png`, `.fig`, `.pdf`, `.svg` are either photos, vector or documents and we will not be concerned about them so we remove them. We  focus here on the first session on the index:

In [47]:
not_data_formats = ['.jpg', '.png', '.pdf', '.svg', '.fig', '.py']

subject = 'YMV01'
date = '170818'
session = f"{subject}_{date}"
session_path = session_path_dic[session]

format_list = list({p.suffix for p in session_path.rglob('*') if not p.is_dir()})
format_list.sort()
format_list = [p for p in format_list if p not in not_data_formats]
pprint(format_list, compact=True)

['', '.1', '.dat', '.eeg', '.json', '.log', '.mat', '.npy', '.nrs', '.nwb',
 '.pkl', '.tsv', '.xml']


The output should be something like this:

    ['', '.1', '.dat', '.eeg', '.json', '.log', '.mat', '.npy', '.nrs',
    '.pkl', '.tsv', '.xml']

The goal of this document is to explore the data available on the rest of the formats and we will do so the following sections. Meanwhile, for orientation purposes, here is a brief description of the available formats and the files associated with them

1. First we have the format '.l' which are actually two formats `.res.1` and `.clu.1`. These are plain files related to the neuroscope sorting format.

2. Then we have the typical '.dat' and '.egg' formats that account for the raw data and the local field potential respectively

3. The `.json` seem to be associated with hidden files corresponding to the `.phy` format. This is related to spike sorting.

4. The `.log` extension is the log file that corresponds to the `phy` program.

5. There is a variety of `.mat` files:

6. There is a varety of `.npy` files.

7. `.nrs`

8. `.pkl` pickled file

9. `.tsv` tabular separated data.

10. `.xml` an xml file



# Neuroscope res and clu
These files have a name ofr hte format `{session}.res` and `{session}.clu`. Those should be the keys of the 
following dics

In [36]:
sorting_files_dic = {p.stem:p for p in session_path.rglob('*') if p.suffix == '.1'}
sorting_files_dic.keys()

dict_keys(['YMV01_170818.res', 'YMV01_170818.clu'])

These are plain text files and can be opened with pandas as a data frame

In [51]:
clu_file_name = f"{session}.clu"
res_file_name = f"{session}.res"

clu_df = pd.read_csv(sorting_files_dic[clu_file_name], header=None, names=['unit'])
res_df = pd.read_csv(sorting_files_dic[res_file_name], header=None, names=['times'])
res_df.shape, clu_df.shape

((1743849, 1), (1743850, 1))

The files should have the same shape. As mentioned those are related to spike sorting. `.clu` contains the units and `.res` the times.
We can concatenat them to have the associated ready

In [54]:
pd.concat([clu_df, res_df], axis=1).sample(n=5)

,unit,times
574500,17,43442222.0
1710774,38,144753785.0
685668,31,53373095.0
1400552,47,121329513.0
1355614,0,117298280.0


# Json files

In [57]:
json_files_dic= {p.stem:p for p in session_path.rglob('*') if p.suffix == '.json'}
json_files_dic

{'new_cluster_id': PosixPath('/home/heberto/globus_data/SenzaiY/YMV01/YMV01_170818/.phy/new_cluster_id.json'),
 'metadata': PosixPath('/home/heberto/globus_data/SenzaiY/YMV01/YMV01_170818/.phy/joblib/phycontrib/template/gui/_get_correlograms/6c2dff01edb0cab33041220e67c54f04/metadata.json')}

These files correspond to some meta data of the `phy` software

# Mat files
Let's gather all the mat files

In [59]:
mat_files_dic = {p.stem:p for p in session_path.iterdir() if p.suffix=='.mat'}
mat_files_dic.keys()

dict_keys(['YMV01_170818_wavelet_REM_8_300Hz', 'YMV01_170818.StatePlotMaterials', 'rez', 'cell_metrics', 'YMV01_170818_wavelet_REM_8_300Hz--Whiten', 'session', 'YMV01_170818--InterpUpDownLFP_params', 'YMV01_170818_wavelet_WAKE_8_300Hz--Whiten', 'YMV01_170818_wavelet_WAKE_8_300Hz', 'YMV01_170818-UnitPhaseMod', 'YMV01_170818_UnitFeature', 'YMV01_170818.chanCoords.channelInfo', 'YMV01_170818.waveform_filter_metrics.cellinfo', 'YMV01_170818.eegstates', 'YMV01_170818-MonoSynConvClick', 'YMV01_170818.SlowWaves.events', 'autoclusta_params', 'YMV01_170818--LFPbasedLayer', 'YMV01_170818.mono_res.cellinfo', 'YMV01_170818.SleepScoreLFP.LFP', 'YMV01_170818.SleepState.states', 'YMV01_170818--InterpDownLFP_params', 'YMV01_170818_wavelet_NREM_8_300Hz--Whiten', 'YMV01_170818.SleepScoreMetrics.LFP', 'depthsort_parameter_1', 'meanWaveforms', 'YMV01_170818.spikes.cellinfo', 'YMV01_170818-DownUpAlignedLFP-CSD', 'YMV01_170818.EMGFromLFP.LFP', 'YMV01_170818.session', 'chanMap', 'YMV01_170818_meanWaveforms',

In [ ]:
'cell_metrics'

In [61]:
file_name = 'cell_metrics'
mat_file_path = mat_files_dic[file_name]
try:
    mat_file = loadmat(mat_file_path)
except NotImplementedError:
    mat_file = mat73.loadmat(mat_file_path, use_attrdict=True)
print(mat_file_path.name, type(mat_file))
print(mat_file.keys())

cell_metrics.mat <class 'mat73.AttrDict'>
dict_keys(['cell_metrics'])


In [60]:
for file_path in mat_files_dic.values():
    try:
        mat_file = loadmat(file_path)
    except NotImplementedError:
        mat_file = mat73.loadmat(file_path, use_attrdict=True)
    print(file_path.name, type(mat_file))
    print(mat_file.keys())

YMV01_170818_wavelet_REM_8_300Hz.mat <class 'dict'>
dict_keys(['__header__', '__version__', '__globals__', 'fmax', 'fmin', 'powMean', 'powStd', 'waveMean', 'waveStd', 'scalefAll', 'f'])
YMV01_170818.StatePlotMaterials.mat <class 'dict'>
dict_keys(['__header__', '__version__', '__globals__', 'StatePlotMaterials'])
rez.mat <class 'mat73.AttrDict'>
dict_keys(['rez'])
cell_metrics.mat <class 'mat73.AttrDict'>
dict_keys(['cell_metrics'])
YMV01_170818_wavelet_REM_8_300Hz--Whiten.mat <class 'dict'>
dict_keys(['__header__', '__version__', '__globals__', 'fmax', 'fmin', 'powMean', 'powStd', 'waveMean', 'waveStd', 'scalefAll', 'f'])
session.mat <class 'mat73.AttrDict'>
dict_keys(['session'])
YMV01_170818--InterpUpDownLFP_params.mat <class 'dict'>
dict_keys(['__header__', '__version__', '__globals__', 'InterpUpDwonLFP_params'])
YMV01_170818_wavelet_WAKE_8_300Hz--Whiten.mat <class 'dict'>
dict_keys(['__header__', '__version__', '__globals__', 'fmax', 'fmin', 'powMean', 'powStd', 'waveMean', 'waveS

# Numpy files

In [11]:
numpy_files_dic = {p.stem:p for p in session.rglob('*') if p.suffix == '.npy'}
numpy_files_dic.keys()

dict_keys(['templates_ind', 'spike_times', 'templates', 'pc_feature_ind', 'whitening_mat_inv', 'similar_templates', 'spike_clusters', 'template_features', 'spike_templates', 'template_feature_ind', 'amplitudes', 'channel_map', 'pc_features', 'channel_positions', 'whitening_mat'])

Let's the spike_times file to explore

In [16]:
numpy_file = np.load(numpy_files_dic['spike_times'])
numpy_file.shape

(1743849, 1)

In [23]:
numpy_file = np.load(numpy_files_dic['amplitudes'])
numpy_file.shape

(1743849, 1)

In [24]:
numpy_file = np.load(numpy_files_dic['channel_map'])
numpy_file.shape

(64, 1)

In [20]:
numpy_file = np.load(numpy_files_dic['spike_clusters'])
np.unique(numpy_file)

array([  0,   1,   4,   5,   9,  10,  11,  15,  17,  18,  19,  20,  22,
        23,  25,  28,  30,  32,  34,  37,  39,  40,  42,  46,  47,  48,
        50,  51,  52,  53,  54,  59,  60,  64,  65,  67,  70,  71,  73,
        74,  76,  79,  80,  81,  82,  85,  86,  88,  89,  90,  92,  93,
        94,  99, 101, 103, 108, 109, 110, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 126, 128, 129, 130, 131, 132,
       133, 134, 135, 136, 139, 140, 141, 142, 143, 144, 145, 147, 148,
       149, 150, 151, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162,
       163, 164, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177,
       178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 190, 191,
       192, 193, 194, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205,
       206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218,
       219, 220, 221, 222, 223, 224, 225, 226, 228, 229, 230, 231, 232,
       233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 244, 24

In [15]:
numpy_file = np.load(numpy_files_dic['templates'])
numpy_file.shape

(384, 44, 64)

# NRS

# Pickled

# TSV - Tabular separated file

# XML